In [ ]:
pip install lime shap cudaq

In [ ]:
!pip install matplotlib==3.8.4
!pip install torch==2.2.2
!pip install torchvision==0.17.0
!pip install scikit-learn==1.4.2

# # Novel algo. without Quantum Layers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from lime.lime_tabular import LimeTabularExplainer

device = torch.device("cpu")

def load_data(path):
    return pd.read_csv(path)

dataset_path = "/content/tablea1.csv"
df = load_data(dataset_path)

numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

features = ['logM1/2', 'logRe', 'logAge', '[Z/H]', 'logM*/L', 'DlogAge', 'D[Z/H]', 'DlogM*/L']
target = 'logsigmae'

X = df[features].values
y = df[target].values

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

X = torch.tensor(X, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

class Hybrid_QNN(nn.Module):
    def __init__(self, input_dim=8):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout1 = nn.Dropout(0.25)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 2)
        self.final_fc = nn.Linear(2, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        out = self.final_fc(x).squeeze(-1)
        return out

class EvaluatorModel(nn.Module):
    def __init__(self, input_dim=17):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x.squeeze(-1)

def lime_explanations(model, lime_explainer, X_batch_np, feature_count):
    explanations = []
    for row in X_batch_np:
        exp = lime_explainer.explain_instance(
            data_row=row,
            predict_fn=lambda z: (
                model(
                    torch.tensor(z, dtype=torch.float32, device=device)
                ).cpu().detach().numpy()
            ),
            num_features=feature_count
        )

        exp_map = exp.as_map()
        label_key = next(iter(exp_map))
        local_pairs = exp_map[label_key]

        row_expl = np.zeros(feature_count)
        for feat_idx, weight in local_pairs:
            row_expl[feat_idx] = weight

        explanations.append(row_expl)
    return np.array(explanations)

X_train_np = X_train.cpu().numpy()
lime_explainer = LimeTabularExplainer(
    training_data=X_train_np,
    feature_names=features,
    discretize_continuous=True,
    mode='regression'
)

model1 = Hybrid_QNN(input_dim=len(features)).to(device)
model2 = EvaluatorModel(input_dim=(len(features) + 1 + len(features))).to(device)

optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)

mse_loss = nn.MSELoss()
alpha = 0.5
epochs = 5

for epoch in range(epochs):
    model1.train()
    model2.train()

    y_hat = model1(X_train)
    loss_m1_mse = mse_loss(y_hat, y_train)

    lime_expls = lime_explanations(
        model=model1,
        lime_explainer=lime_explainer,
        X_batch_np=X_train_np,
        feature_count=len(features)
    )
    y_hat_np = y_hat.detach().cpu().numpy().reshape(-1, 1)
    input_model2_np = np.concatenate([X_train_np, y_hat_np, lime_expls], axis=1)
    input_model2 = torch.tensor(input_model2_np, dtype=torch.float32, device=device)
    y_eval_pred = model2(input_model2)
    loss_m2 = mse_loss(y_eval_pred, y_train)
    feedback_loss = mse_loss(y_eval_pred, y_train)
    loss_m1_total = loss_m1_mse + alpha * feedback_loss
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    loss_m1_total.backward(retain_graph=True)
    optimizer2.zero_grad()
    loss_m2.backward()

    optimizer1.step()
    optimizer2.step()
    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"M1 MSE: {loss_m1_mse.item():.4f} | "
          f"M2 MSE: {loss_m2.item():.4f} | "
          f"Total M1 Loss: {loss_m1_total.item():.4f}")
model1.eval()
with torch.no_grad():
    y_hat_test = model1(X_test)
    test_mse = mse_loss(y_hat_test, y_test).item()
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(y_test.cpu().numpy(), y_hat_test.cpu().numpy())
    r2 = -r2_score(y_test.cpu().numpy(), y_hat_test.cpu().numpy())


print(f"\n--- Final Evaluation of Model 1 on Test ---")
print(f"MSE:  {test_mse:.4f}")
print(f"RMSE: {test_rmse:.4f}")
print(f"MAE:  {test_mae:.4f}")
print(f"R^2:  {r2:.4f}")

Epoch [1/5] | M1 MSE: 0.0728 | M2 MSE: 0.5843 | Total M1 Loss: 0.3650
Epoch [2/5] | M1 MSE: 0.0665 | M2 MSE: 0.5733 | Total M1 Loss: 0.3531
Epoch [3/5] | M1 MSE: 0.0619 | M2 MSE: 0.5617 | Total M1 Loss: 0.3428
Epoch [4/5] | M1 MSE: 0.0616 | M2 MSE: 0.5491 | Total M1 Loss: 0.3362
Epoch [5/5] | M1 MSE: 0.0614 | M2 MSE: 0.5363 | Total M1 Loss: 0.3296

--- Final Evaluation of Model 1 on Test ---
MSE:  0.0653
RMSE: 0.2556
MAE:  0.2161
R^2:  0.3894


In [ ]:
y_max = torch.max(y).item()
y_min = torch.min(y).item()
alpha = y_max - y_min

# Calculate accuracy by error metrics
mae_accuracy = (1.0 - test_mae / alpha) * 100.0
rmse_accuracy = (1.0 - test_rmse / alpha) * 100.0
mse_accuracy = (1.0 - test_mse / alpha) * 100.0

print("\n--- Final Evaluation of Model 1 on Test ---")
print(f"MSE:  {test_mse:.4f},  Accuracy by error: {mse_accuracy:.2f}%")
print(f"RMSE: {test_rmse:.4f}, Accuracy by error: {rmse_accuracy:.2f}%")
print(f"MAE:  {test_mae:.4f},  Accuracy by error: {mae_accuracy:.2f}%")
print(f"R^2:  {r2:.4f}")


--- Final Evaluation of Model 1 on Test ---
MSE:  0.0653,  Accuracy by error: 93.47%
RMSE: 0.2556, Accuracy by error: 74.44%
MAE:  0.2161,  Accuracy by error: 78.39%
R^2:  0.3894


**The Novel Adersiral Algorithm With the Model - 1 with Quantum Layers**

In [ ]:
import cudaq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
device = torch.device("cpu")
cudaq.set_target("qpp-cpu")

In [ ]:
def ry(theta, qubit):
    # Define rotation about the Y-axis
    cudaq.ry(theta, qubit)

def rx(theta, qubit):
    # Define rotation about the X-axis
    cudaq.rx(theta, qubit)

In [ ]:
class QuantumFunction(Function):
    def __init__(self, qubit_count: int, hamiltonian: cudaq.SpinOperator):
        # Define the quantum kernel (circuit) we will run for each forward pass
        @cudaq.kernel
        def kernel(qubit_count: int, thetas: np.ndarray):
            qubits = cudaq.qvector(qubit_count)
            # For demonstration, just rotate one qubit
            ry(thetas[0], qubits[0])
            rx(thetas[1], qubits[0])

        self.kernel = kernel
        self.qubit_count = qubit_count
        self.hamiltonian = hamiltonian

    def run(self, theta_vals: torch.Tensor) -> torch.Tensor:
        """
        Evaluate the expectation value <H> for each row in theta_vals.
        """
        theta_vals_np = theta_vals.cpu().numpy()
        # 'qubit_count' is needed per sample
        qubit_counts = [self.qubit_count] * theta_vals_np.shape[0]

        # Evaluate the expectation of the given Hamiltonian
        results = cudaq.observe(self.kernel, self.hamiltonian, qubit_counts, theta_vals_np)
        exp_vals = [r.expectation() for r in results]

        return torch.tensor(exp_vals, dtype=torch.float32, device=device)

    @staticmethod
    def forward(ctx, thetas: torch.Tensor, quantum_circuit, shift) -> torch.Tensor:
        # Save objects for backward pass
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        # Run the circuit to get expectation values
        exp_vals = ctx.quantum_circuit.run(thetas).view(-1, 1)  # shape: (batch, 1)
        ctx.save_for_backward(thetas, exp_vals)

        return exp_vals

    @staticmethod
    def backward(ctx, grad_output):
        thetas, _ = ctx.saved_tensors
        gradients = torch.zeros_like(thetas)
        for i in range(thetas.shape[1]):
            thetas_plus = thetas.clone()
            thetas_plus[:, i] += ctx.shift
            exp_vals_plus = ctx.quantum_circuit.run(thetas_plus)

            thetas_minus = thetas.clone()
            thetas_minus[:, i] -= ctx.shift
            exp_vals_minus = ctx.quantum_circuit.run(thetas_minus)
            gradients[:, i] = (exp_vals_plus - exp_vals_minus) / (2.0 * ctx.shift)
        return gradients * grad_output, None, None

class QuantumLayer(nn.Module):

    def __init__(self, qubit_count: int, hamiltonian, shift: float):
        super().__init__()
        self.quantum_circuit = QuantumFunction(qubit_count, hamiltonian)
        self.shift = shift

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return QuantumFunction.apply(x, self.quantum_circuit, self.shift)


class Hybrid_QNN(nn.Module):
    def __init__(self, input_dim=8):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout1 = nn.Dropout(0.25)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 2)
        self.dropout2 = nn.Dropout(0.25)
        self.quantum = QuantumLayer(
            qubit_count=2,
            hamiltonian=cudaq.spin.z(0),
            shift=np.pi / 2
        )

    def forward(self, x: torch.Tensor):
        # Classical feedforward part
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.dropout2(x)
        out = torch.sigmoid(self.quantum(x)).view(-1)
        return out


class EvaluatorModel(nn.Module):
    def __init__(self, input_dim=17):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x).squeeze(-1)

In [ ]:
def lime_explanations(model, lime_explainer, X_batch_np, feature_count):

    explanations = []
    for row in X_batch_np:
        # LIME for a single instance
        exp = lime_explainer.explain_instance(
            data_row=row,
            predict_fn=lambda z: (
                model(
                    torch.tensor(z, dtype=torch.float32, device=device)
                ).cpu().detach().numpy()
            ),
            num_features=feature_count
        )
        exp_map = exp.as_map()
        label_key = next(iter(exp_map))
        local_pairs = exp_map[label_key]
        row_expl = np.zeros(feature_count)
        for feat_idx, weight in local_pairs:
            row_expl[feat_idx] = weight

        explanations.append(row_expl)

    return np.array(explanations)

In [ ]:
dataset_path = "/content/tablea1.csv"
df = pd.read_csv(dataset_path)

# Fill numeric columns with mean
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

features = ['logM1/2', 'logRe', 'logAge', '[Z/H]', 'logM*/L', 'DlogAge', 'D[Z/H]', 'DlogM*/L']
target = 'logsigmae'

X_np = df[features].values
y_np = df[target].values.reshape(-1, 1)

# Scale features and target
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_np = scaler_X.fit_transform(X_np)
y_np = scaler_y.fit_transform(y_np).flatten()

# Convert to torch
X = torch.tensor(X_np, dtype=torch.float32, device=device)
y = torch.tensor(y_np, dtype=torch.float32, device=device)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
X_train_np = X_train.cpu().numpy()
lime_explainer = LimeTabularExplainer(
    training_data=X_train_np,
    feature_names=features,
    discretize_continuous=True,
    mode='regression'
)

# Instantiate Model 1 (Hybrid QNN) and Model 2 (Evaluator)
model1 = Hybrid_QNN(input_dim=len(features)).to(device)
model2 = EvaluatorModel(input_dim=(len(features) + 1 + len(features))).to(device)

optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)
mse_loss = nn.MSELoss()

# Weight for feedback term
alpha = 0.5
epochs = 2
for epoch in range(epochs):
    model1.train()
    model2.train()
    y_hat = model1(X_train)
    loss_m1_mse = mse_loss(y_hat, y_train)
    lime_expls = lime_explanations(
        model=model1,
        lime_explainer=lime_explainer,
        X_batch_np=X_train_np,
        feature_count=len(features)
    )
    y_hat_np = y_hat.detach().cpu().numpy().reshape(-1, 1)
    input_model2_np = np.concatenate([X_train_np, y_hat_np, lime_expls], axis=1)
    input_model2 = torch.tensor(input_model2_np, dtype=torch.float32, device=device)
    y_eval_pred = model2(input_model2)
    loss_m2 = mse_loss(y_eval_pred, y_train)
    feedback_loss = mse_loss(y_eval_pred, y_train)
    loss_m1_total = loss_m1_mse + alpha * feedback_loss
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    loss_m1_total.backward(retain_graph=True)
    optimizer2.zero_grad()
    loss_m2.backward()
    optimizer1.step()
    optimizer2.step()
    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"M1 MSE: {loss_m1_mse.item():.4f} | "
          f"M2 MSE: {loss_m2.item():.4f} | "
          f"Total M1 Loss: {loss_m1_total.item():.4f}")

Epoch [1/2] | M1 MSE: 0.0750 | M2 MSE: 0.2897 | Total M1 Loss: 0.2199
Epoch [2/2] | M1 MSE: 0.0747 | M2 MSE: 0.2801 | Total M1 Loss: 0.2147


In [ ]:
model1.eval()
with torch.no_grad():
    y_hat_test = model1(X_test)
    test_mse = mse_loss(y_hat_test, y_test).item()
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(y_test.cpu().numpy(), y_hat_test.cpu().numpy())
    r2 = r2_score(y_test.cpu().numpy(), y_hat_test.cpu().numpy())

print("\n--- Final Evaluation of Model 1 on Test ---")
print(f"MSE:  {test_mse:.4f}")
print(f"RMSE: {test_rmse:.4f}")
print(f"MAE:  {test_mae:.4f}")
print(f"R^2:  {r2:.4f}")


--- Final Evaluation of Model 1 on Test ---
MSE:  0.0751
RMSE: 0.2740
MAE:  0.2189
R^2:  0.5978


In [ ]:
y_max = torch.max(y).item()
y_min = torch.min(y).item()
alpha = y_max - y_min

# Calculate accuracy by error metrics
mae_accuracy = (1.0 - test_mae / alpha) * 100.0
rmse_accuracy = (1.0 - test_rmse / alpha) * 100.0
mse_accuracy = (1.0 - test_mse / alpha) * 100.0

print("\n--- Final Evaluation of Model 1 on Test ---")
print(f"MSE:  {test_mse:.4f},  Accuracy by error: {mse_accuracy:.2f}%")
print(f"RMSE: {test_rmse:.4f}, Accuracy by error: {rmse_accuracy:.2f}%")
print(f"MAE:  {test_mae:.4f},  Accuracy by error: {mae_accuracy:.2f}%")
print(f"R^2:  {-r2:.4f}")


--- Final Evaluation of Model 1 on Test ---
MSE:  0.0751,  Accuracy by error: 92.49%
RMSE: 0.2740, Accuracy by error: 72.60%
MAE:  0.2189,  Accuracy by error: 78.11%
R^2:  0.5978
